In [122]:
print("Assignment 2, Part 2, Implementation of Decision Tree using ID3 Algorithm")
#Authors
#Meetika Sharma : MXS173530
#Yash Pradhan : YPP170130

Assignment 2, Part 2, Implementation of Decision Tree using ID3 Algorithm


In [123]:
from math import log
import operator

In [124]:
#function: getTrainingDataset() to get the training dataset
def getTrainingDataset():
    import csv
    with open(training_path, 'r') as instances:
        myreader = csv.reader(instances)
        #get header and data instances from the csv
        headers = next(myreader, None)
        training_examples = [list(map(int,rec)) for rec in csv.reader(instances, delimiter=',')]
    #the last header is the class label, hence strip it
    headers = headers[:-1]
    
    #print("Headers:", headers)
    #print("Training Examples:", training_examples)
    return training_examples, headers

In [125]:
#function: getTestingDataset() to get the testing dataset
def getTestingDataset():
    import csv
    with open(testing_path, 'r') as instances:
        myreader = csv.reader(instances)
        #get header and data instances from the csv
        headers = next(myreader, None)
        testing_examples = [list(map(int,rec)) for rec in csv.reader(instances, delimiter=',')]
    #the last header is the class label, hence strip it
    headers = headers[:-1]
    
    #print("Headers:", headers)
    #print("Testing Examples:", testing_examples)
    return testing_examples, headers

In [126]:
def getValidationDataset():
    import csv
    with open(validation_path, 'r') as instances:
        myreader = csv.reader(instances)
        #get header and data instances from the csv
        headers = next(myreader, None)
        validation_examples = [list(map(int,rec)) for rec in csv.reader(instances, delimiter=',')]
    #the last header is the class label, hence strip it
    headers = headers[:-1]
    
    #print("Headers:", headers)
    #print("Testing Examples:", testing_examples)
    return validation_examples, headers

In [127]:
#We'll use the formula to find the Entropy
def findEntropyAtNode(instances):
    totalInstances = len(instances)
    #keep a track of postive and negative labels
    countClass = {}
    for instance in instances:
        currentClass = instance[-1]
        if currentClass not in countClass.keys(): countClass[currentClass] = 0
        countClass[currentClass] += 1
    entropy = 0.0
    for key in countClass:
        probability = float(countClass[key])/totalInstances
        entropy -= probability * log(probability, 2)
    return entropy

In [128]:
def findBestAttribute(dataset):
    totalFeatures = len(dataset[0]) - 1
    parentEntropy = findEntropyAtNode(dataset)
    maxInformationGain = 0.0
    indexBestFeature = -1
    
    #iterate over all features to find the best feature
    # i.e. the one with the maximum information gain
    for i in range(totalFeatures):
        featureList = [instance[i] for instance in dataset]
        distinctValues = set(featureList)
        entropy = 0.0
        
        #calculate the number of instances that belong to each class based on this attribute
        for value in distinctValues:
            subset = splitDataset(dataset, i, value)
            probability = len(subset)/float(len(dataset))
            entropy += probability * findEntropyAtNode(subset)
        
        informationGain = parentEntropy - entropy
        
        if(informationGain>maxInformationGain):
            maxInformationGain = informationGain
            indexBestFeature = i
    return indexBestFeature

In [129]:
#In one of the termination condition we label the node as the majority class
#the following function returns majority class for current node
def getMajorityClass(classList):
    classCount = {}
    for label in classList:
        if label not in classCount.keys(): classCount[label] = 0
        classCount[label] += 1
        
    #sort to get the class with max counts   
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [130]:
#to split the dataset after finding the best attribute, strip the selected attribute axis
def splitDataset(dataset, selectedAttribute, value):
    retDataset = []
    for instance in dataset:
        if instance[selectedAttribute] == value:
            subDataset = instance[:selectedAttribute]  # chop out axis used for splitting
            subDataset.extend(instance[selectedAttribute + 1:])
            retDataset.append(subDataset)
    return retDataset

In [131]:
#The ID3 Algotithm to generate decision tree
def createTree(dataset,headers):
    classList = [instance[-1] for instance in dataset]
    
    #splitting stops when all instances belong to the same class
    if classList.count(classList[0]) == len(classList):
        return classList[0]  
    #another criteria is that, there are no more attributes to split upon
    if len(dataset[0]) == 1:  
        return getMajorityClass(classList)
    
    #Finding the best attribute to split upon
    bestFeat = findBestAttribute(dataset)
    bestFeatLabel = headers[bestFeat]

    #build a tree recursively
    decisionTree = {bestFeatLabel: {}}
    
    del (headers[bestFeat])
    
    featValues = [instance[bestFeat] for instance in dataset]
    uniqueVals = set(featValues)
    
    for value in uniqueVals:
        subHeaders = headers[:]  
        decisionTree[bestFeatLabel][value] = createTree(splitDataset(dataset, bestFeat, value), subHeaders)
        decisionTree[bestFeatLabel]['majority_class'] = int(getMajorityClass(classList))
        
    return decisionTree

In [132]:
#classify an instance
def classify(inputTree, featureLabels, instanceVector):
    #print("Let's classify", instanceVector)
    
    # find the attribute on which split has been performed
    for key in inputTree:
        #print ("key: "+ key +", value: "+str(inputTree[key]))
        #print("Current Splitting Attribute:", key)
        innerpart = inputTree[key]
        #print("Inner Part:", str(innerpart))
    
    # find the value that this instance holds for the splitting attribute
    index = featureLabels.index(key)
    #print("Index of attribute", key, "is ", index)
    value = instanceVector[index]
    #print("Value for attribute", key, "is ", value)
    
    innervalue = innerpart[value]
    #print(innervalue)
    # In the tree find what holds for Attribute Ai and its corresponding value of instance
    
    # If there is a node further on, apply recursively till you reach the root node
    if isinstance(innervalue, dict):
        #print(innervalue)
        classLabel = classify(innervalue, featureLabels, instanceVector)
    else:
    # If it is a class label, it means we have reached the root node and hence ready for prediction
    # the class label is the prediction, return it to the driver function
        classLabel = innervalue
    
    return classLabel

In [133]:
#getting accuracy on data
def calculateAccuracy(decisiontree, dataset, labels):
    total_instances = len(dataset)
    num_correct_predictions = 0
    
    for instance in dataset:
        predicted_class_label = classify(decisiontree, labels, instance[:-1])
        if(predicted_class_label == instance[-1]):
            num_correct_predictions += 1
            
    accuracy = num_correct_predictions/total_instances
    return accuracy

In [134]:
#print the tree
def printTree(tree, d=0):
    for attribute in tree:
        options = tree[attribute]
        for values in options:
            subtree = options[values]
            if(isinstance(subtree, dict)):
                print(" |" * d,attribute, "=", values, ":")
                printTree(subtree, d+1)
            else:
                if(values!='majority_class'):
                    print(" |" * d,attribute, "=", values, ":", subtree)

In [135]:
def getPrunedTree(tree):
    #pruning logic yet to be implemented
    pruned_tree = tree
    return pruned_tree

In [136]:
def getTotalNodes(tree):
    str_tree = str(tree)
    return str_tree.count("{") + 1

In [137]:
def getNumLeaves(tree):
    N = getTotalNodes(tree)
    return (N+1)/2

In [138]:
#get input from user
training_path = input("Enter path of training data set: ")
testing_path = input("Enter path of testing data set: ")
validation_path = input("Enter path of validation data set: ")
pruning_factor = input("Enter pruning factor: ")
    
# collect data
dataset, headers = getTrainingDataset()

Enter path of training data set: training_set.csv
Enter path of testing data set: test_set.csv
Enter path of validation data set: validation_set.csv
Enter pruning factor: 0.1


In [139]:
#build the decision tree
decisiontree = createTree(dataset, headers)
print("Decision Tree generated by ID3 Algo.")
printTree(decisiontree)

Decision Tree generated by ID3 Algo.
 XO = 0 :
 | XM = 0 :
 | | XF = 0 :
 | | | XB = 0 :
 | | | | XG = 0 : 0
 | | | | XG = 1 :
 | | | | | XD = 0 :
 | | | | | | XS = 0 : 0
 | | | | | | XS = 1 :
 | | | | | | | XC = 0 : 1
 | | | | | | | XC = 1 :
 | | | | | | | | XH = 0 : 0
 | | | | | | | | XH = 1 : 1
 | | | | | XD = 1 :
 | | | | | | XE = 0 : 0
 | | | | | | XE = 1 :
 | | | | | | | XK = 0 : 0
 | | | | | | | XK = 1 : 1
 | | | XB = 1 :
 | | | | XD = 0 : 0
 | | | | XD = 1 :
 | | | | | XI = 0 : 0
 | | | | | XI = 1 :
 | | | | | | XG = 0 : 1
 | | | | | | XG = 1 : 0
 | | XF = 1 : 0
 | XM = 1 :
 | | XB = 0 :
 | | | XD = 0 :
 | | | | XG = 0 :
 | | | | | XF = 0 : 0
 | | | | | XF = 1 :
 | | | | | | XJ = 0 :
 | | | | | | | XN = 0 : 1
 | | | | | | | XN = 1 :
 | | | | | | | | XE = 0 :
 | | | | | | | | | XK = 0 : 0
 | | | | | | | | | XK = 1 : 1
 | | | | | | | | XE = 1 : 0
 | | | | | | XJ = 1 :
 | | | | | | | XC = 0 :
 | | | | | | | | XT = 0 :
 | | | | | | | | | XL = 0 :
 | | | | | | | | | | XE = 0 :
 | | 

In [140]:
training_dataset, training_headers = getTrainingDataset()
training_accuracy = calculateAccuracy(decisiontree, training_dataset, training_headers)

testing_dataset, testing_headers = getTestingDataset()
testing_accuracy = calculateAccuracy(decisiontree, testing_dataset, testing_headers)

validation_dataset, validation_headers = getValidationDataset()
validation_accuracy = calculateAccuracy(decisiontree, validation_dataset, validation_headers)

print("Pre Pruned Accuracy")
print('-'*25)
print("Number of training instances =", len(training_dataset))
print("Number of training attributes =", len(training_headers))

print("Total number of nodes in the tree =", getTotalNodes(decisiontree))
print("Total number of leaf nodes in the tree =", getNumLeaves(decisiontree))
print("Accuracy of the model on the training dataset =", training_accuracy*100)

print("\nNumber of validation instances =", len(validation_dataset))
print("Number of validation attributes =", len(validation_headers))
print("Accuracy of the model on the validation dataset before pruning = ", validation_accuracy*100)

print("\nNumber of testing instances =", len(testing_dataset))
print("Number of testing attributes =", len(testing_headers))
print("Accuracy of the model on the testing dataset =", testing_accuracy*100)

Pre Pruned Accuracy
-------------------------
Number of training instances = 600
Number of training attributes = 20
Total number of nodes in the tree = 275
Total number of leaf nodes in the tree = 138.0
Accuracy of the model on the training dataset = 100.0

Number of validation instances = 2000
Number of validation attributes = 20
Accuracy of the model on the validation dataset before pruning =  75.9

Number of testing instances = 2000
Number of testing attributes = 20
Accuracy of the model on the testing dataset = 75.85


In [141]:
#Pruning the tree
prunedtree = getPrunedTree(decisiontree)

training_accuracy = calculateAccuracy(decisiontree, training_dataset, training_headers)
testing_accuracy = calculateAccuracy(decisiontree, testing_dataset, testing_headers)
validation_accuracy = calculateAccuracy(decisiontree, validation_dataset, validation_headers)

print("Post Pruned Accuracy")
print('-'*25)
print("Number of training instances =", len(training_dataset))
print("Number of training attributes =", len(training_headers))

print("Total number of nodes in the tree =", getTotalNodes(decisiontree))
print("Total number of leaf nodes in the tree =", getNumLeaves(decisiontree))
print("Accuracy of the model on the training dataset =", training_accuracy*100)

print("\nNumber of validation instances =", len(validation_dataset))
print("Number of validation attributes =", len(validation_headers))
print("Accuracy of the model on the validation dataset before pruning = ", validation_accuracy*100)

print("\nNumber of testing instances =", len(testing_dataset))
print("Number of testing attributes =", len(testing_headers))
print("Accuracy of the model on the testing dataset =", testing_accuracy*100)

Post Pruned Accuracy
-------------------------
Number of training instances = 600
Number of training attributes = 20
Total number of nodes in the tree = 275
Total number of leaf nodes in the tree = 138.0
Accuracy of the model on the training dataset = 100.0

Number of validation instances = 2000
Number of validation attributes = 20
Accuracy of the model on the validation dataset before pruning =  75.9

Number of testing instances = 2000
Number of testing attributes = 20
Accuracy of the model on the testing dataset = 75.85
